In [ ]:
import numpy as np

In [ ]:
# metrics = ['clr', 'manhattan', 'euclidean', 'cosine'] + [f'minkowski_{str(p)}' for p in np.array([0.5, 1, 2, 3, 4, 5])]
# metrics

In [ ]:
# manually curated metrics + metrics refered to in the documentation
all_doc_metrics = ['angular', 'clr'] + ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan'] + ['nan_euclidean'] + ['braycurtis', 'canberra', 'chebyshev', 'correlation', 'dice', 'hamming', 'jaccard', 'kulsinski', 'mahalanobis', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule']
# manually curated metrics + metrics refered to in metric parameter ValueError (sklearn documentation is likely not updated)
all_metrics = ['angular', 'clr'] + ['euclidean', 'l2', 'l1', 'manhattan', 'cityblock', 'braycurtis', 'canberra', 'chebyshev', 'correlation', 'cosine', 'dice', 'hamming', 'jaccard', 'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto', 'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean', 'yule', 'wminkowski', 'nan_euclidean', 'haversine']
# metrics that covert data to boolean (essentially destroying all information for our data) (20 clusters are always produced with the same exact size)
boolean_metrics = [
'dice',
'jaccard',
'rogerstanimoto',
'russellrao',
'sokalmichener',
'sokalsneath',
'yule',
]
orig_metrics = ['clr', 'manhattan', 'euclidean', 'cosine'] + [f'minkowski_{str(p)}' for p in np.array([0.5, 1, 2, 3, 4, 5])]
# metrics = [sys.argv[1]]
metrics = [m for m in all_metrics if m not in orig_metrics + boolean_metrics and m[: len('minkowski')] != 'minkowski']
metrics

In [ ]:
len([m for m in all_metrics if m not in boolean_metrics])

In [ ]:
[m for m in all_metrics + orig_metrics if m not in boolean_metrics]

In [ ]:
metrics = [m for m in all_metrics + orig_metrics if m not in boolean_metrics] + ['clr_lev']
metrics

In [ ]:
len(metrics)

In [ ]:
nns = np.arange(2, 13, 1)
nns

In [ ]:
datasets = ['microarray', 'rna_seq']

In [ ]:
partition_types = ['EXP', 'NC', 'TNC']

In [ ]:
for partition_type in partition_types:
    for dataset in datasets:
        for metric in metrics:
            for nn in nns:

                sbatch_str = f'''#!/bin/bash
    #SBATCH --job-name=run_scan_{metric}_{nn}_{dataset}_{partition_type}
    #SBATCH --account=pi-apturkew
    #SBATCH --partition=broadwl
    #SBATCH --ntasks-per-node=1
    #SBATCH --cpus-per-task=20
    #SBATCH --mem-per-cpu=2000
    ##SBATCH --output=myjob_%j.out      # Standard output log
    ##SBATCH --error=myjob_%j.err       # Standard error log
    #SBATCH --mail-type=ALL
    #SBATCH --mail-user=bertagna@rcc.uchicago.edu



    source activate tgne_all.env

    cmd="python3 fast_scan_modular_metrics.py {dataset} {metric} {nn} {partition_type}"

    echo $cmd

    eval $cmd
    '''         
                tags = [dataset, metric, nn, partition_type]
                output_file = f'./TGNE/clustering_optimization/{"_".join([str(tag) for tag in tags])}.sbatch'
                with open(output_file, 'w') as f:
                    f.write(sbatch_str)
                    print('FILE CREATED:', output_file)